# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [85]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [86]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [87]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [88]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create tables based on queries in Apache Cassandra.

## Columns within the 'event_datafile_new.csv'.
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font><br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [96]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [97]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [98]:
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)

### Create a table "song_by_session" for the following query:
#### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
#### For table song_by_session, the sessionId is used as a partition key because the query will be filtered by this column. itemInSession is used as a clustering column to ensure the key is unique. 

In [99]:
query = "CREATE TABLE IF NOT EXISTS song_by_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float,"
query = query + "PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [100]:
%%time
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

CPU times: user 2.96 s, sys: 578 ms, total: 3.54 s
Wall time: 6.66 s


#### Do a SELECT to verify that the data have been inserted into each table

In [101]:
%%time
query = "select artist, song, length from song_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print (result.artist, result.song, result.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875
CPU times: user 2.93 ms, sys: 582 µs, total: 3.51 ms
Wall time: 6.28 ms


### Create a table "song_by_user" for the following query:
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### For table song_by_user, the userid and sessionId are used as a composite partition key. This will spread the userid into different nodes, therefore better performance when searching a specific sessionId. itemInSession is used as the clustering column so that each key is unique.

In [102]:
query = "CREATE TABLE IF NOT EXISTS song_by_user"
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,"
query = query + "PRIMARY KEY ((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [103]:
%%time
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

CPU times: user 2.83 s, sys: 632 ms, total: 3.46 s
Wall time: 6.4 s


#### Do a SELECT to verify that the data have been inserted into each table

In [104]:
query = "select artist, song, firstname, lastname from song_by_user WHERE userid = 10 and sessionId = 182"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print (result.artist, result.song, result.firstname, result.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Create a table "user_by_song" for the following query:
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.
#### For user_by_song table, song is used as the partition key because of the query will filter by this column. userid is used as the clustering column to ensure the key is unique. I don't use sessionId here because it doesn't matter how many times each user has listened to a song. 

In [105]:
query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + "(firstname text, lastname text, song text, userid int,"
query = query + "PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [106]:
%%time
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (firstname, lastname, song, userid)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

CPU times: user 2.85 s, sys: 580 ms, total: 3.43 s
Wall time: 6.42 s


#### Do a SELECT to verify that the data have been inserted into each table

In [107]:
query = "select firstname, lastname from user_by_song WHERE song = 'All Hands Against His Own'"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print (result.firstname, result.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [108]:
query = "drop table if exists song_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [109]:
session.shutdown()
cluster.shutdown()